# Lab1: Data Exploration with BigQuery

### Supporting research: Covid19 vaccine and beyond

BigQuery is serverless, highly scalable, and cost-effective cloud data warehouse designed for business agility. BQ enables us to analyze petabytes of data using standard ANSI SQL at blazing-fast speeds, with zero operational overhead. BigQuery ML enables users to create and execute machine learning models within BigQuery using SQL queries. The goal is to democratize machine learning by enabling SQL practitioners to build models using their existing tools and to increase development speed by eliminating the need for data movement.

In this tutorial, you use the peptidic epitope data available as BigQuery Public-Dataset. BigQuery provides many data set for public research for Covid19 and other genomic research. The peptidic epitope dataset is sourced from NIH's IEDB and are hosted by BigQuery.

## Objectives
In this tutorial, you use:
+ BigQuery to explore antigen data for coronavirus
+ Explore antigen dataset to understand protein structure distribution
+ Analyze epitope dataset to investigate spike protein properties
+ Visualize and explore peptides that are most likely to bind with MHC molecule

Jupyter magics are notebook-specific shortcuts that allow you to run commands with minimal syntax. Jupyter notebooks come with many [built-in commands](https://ipython.readthedocs.io/en/stable/interactive/magics.html). The BigQuery client library, `google-cloud-bigquery`, provides a cell magic, `%%bigquery`. The `%%bigquery` magic runs a SQL query and returns the results as a pandas `DataFrame`.

## Step 1: Explore Peptidic Epitope Data
You can create GCP project and leverage public data set which provides free access to many datasets and free query processing. See more information [here](https://cloud.google.com/bigquery/public-data).

For our coronavirus subset of data, let's examine data first:
+ Following query shows information about antigen.
+ Observe specific antigen that might of more interest to our study for coronavirus.
+ Can you find it?


In [ ]:
# Read GCP project id from env.
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
GCP_PROJECT_ID=shell_output[0]
#print("GCP project ID:" + GCP_PROJECT_ID)

In [ ]:
%%bigquery --project $GCP_PROJECT_ID
SELECT * 
FROM `bigquery-public-data.immune_epitope_db.antigen_full_v3`
WHERE organism_name like '%coronavirus%'
LIMIT 10

Visually inspect number of epitopes by antigen name in our dataset.

In [ ]:
%%bigquery epitopes --project $GCP_PROJECT_ID
SELECT
 Antigen_Name,
 sum(number_of_epitopes) AS epitope
FROM `bigquery-public-data.immune_epitope_db.antigen_full_v3`
WHERE organism_name like '%coronavirus%'
GROUP BY Antigen_Name
ORDER BY epitope DESC

In [ ]:
epitopes.plot(kind='bar', x='Antigen_Name', y='epitope');

In [ ]:
# Following query shows epitope detail for those linear peptides whose parent protein
# came from Spike glycoprotein, which is our interests to investigate for possible antigen
# to protect against coronavirus


In [ ]:
%%bigquery --project $GCP_PROJECT_ID
SELECT epitope_iri, Object_Type, Description, Starting_Position,
            Ending_Position, Antigen_Name,Parent_Protein, Organism_Name
FROM `bigquery-public-data.immune_epitope_db.epitope_full_v3` 
WHERE Parent_Protein = 'Spike glycoprotein'
AND organism_name like '%coronavirus%'
LIMIT 10

As you can see from the resulting query, we have a set of peptides generated from parent antigen protein. Now, we need to identify how these peptides bind with HLA molecule.

Following query shows HLA-peptide binding affinity information. The goal is to leverage machine learning models to predict binding affinity for many different and new peptides with HLA MHC Class-I molecules so that testing for vaccine candidates can be prioritized to accelerate solutions.


In [ ]:
%%bigquery --project $GCP_PROJECT_ID
SELECT Reference_iri, Description, Allele_Name,MHC_allele_class, Assay_Group,
       Qualitative_Measure, Quantitative_measurement 
FROM `bigquery-public-data.immune_epitope_db.mhc_ligand_full`
WHERE pubmed_id = 15104671 
LIMIT 3


In our dataset there are multiple of alleles and, for each allele we have epitope binding affinity measure. Lete inspect how epitopes and allele bindings are distributed in the dataset. This is one way to short list possible allele to consider for peptide binding research.

We will use dataframe to hold query result to build our visualization.

In [ ]:
from google.cloud import bigquery
client = bigquery.Client()

In [ ]:
sql = """
SELECT Qualitative_Measure, 
       Allele_Name, 
       count(1) as count 
FROM `bigquery-public-data.immune_epitope_db.mhc_ligand_full`
where organism_name like '%coronavirus%'
GROUP BY  1,2
ORDER BY 3 DESC
"""
df = client.query(sql).to_dataframe()
df.head()

In [ ]:
pivot_table = df.pivot(index='Allele_Name', columns='Qualitative_Measure', values='count')
pivot_table.plot(kind='bar', stacked=True, figsize=(15, 7));

Based on data insepction, it seems like for coronavirus data set, HLA-A-01-01, HLA-A-02-01, HLA-A-02-02, HLA-A-02-03 and HLA-A-020-06 alleles are most likely candidates for MHC-I molecules.

It is important to narrow down focus peptides and HLA binding to speed up testing on most probable vaccine candidate. Lets identify set of peptides that binds well with HLA, you can further narrow it to specific HLA as well. Following qury execution suggest that peptides with length of 9 and 10 are having better binding affinity. We should focus our research on those for now. Of course, more factors, features as well as 3D structure of aplha/beta components of HLA+Peptide bindings are useful for more complex modeling. 

In [ ]:
sql = """
SELECT Qualitative_Measure, 
       length(Description) as Peptide_Length, 
       count(1) as count 
FROM `bigquery-public-data.immune_epitope_db.mhc_ligand_full`
where organism_name like '%coronavirus%'
GROUP BY  1,2
ORDER BY 3 DESC
"""
df = client.query(sql).to_dataframe()
df.head()

In [ ]:
pivot_table = df.pivot(index='Peptide_Length', columns='Qualitative_Measure', values='count')
pivot_table.plot(kind='bar', stacked=False, figsize=(15, 7));

Chart above demonstrate that peptide with length of 9 or 10 mers are most suitable and probable candidates to investigate further for affinity modeling.

### This completes Lab1. Congratulations!